In [25]:
import urllib2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json
from collections import defaultdict

import datetime as dt
import progressbar

import time
from BeautifulSoup import BeautifulSoup

In [26]:


urls = {
    "Star Wars: Episode I The Phantom Menice": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-i-the-phantom-menace",
    "Star Wars: Episode II Attack of the Clones": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-ii-attack-of-the-clones",
    "Star Wars: The Clone Wars (film)": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-the-clone-wars",
    "Star Wars: Episode III Revenge of the Sith": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-iii-revenge-of-the-sith",
    "Solo: A Star Wars Story": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=solo-a-star-wars-story",
    "Rogue One: A Star Wars Story": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=rogue-one-a-star-wars-story",
    "Star Wars: Episode IV A New Hope": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-iv-a-new-hope",
    "Star Wars: Episode V The Empire Strikes Back": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-v-the-empire-strikes-back",
    "Star Wars: Episode VI Return of the Jedi": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-vi-return-of-the-jedi",
    "Star Wars: Episode VII The Force Awakens": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-vii-the-force-awakens",
    "Star Wars: Episode VIII The Last Jedi": "https://www.springfieldspringfield.co.uk/movie_script.php?movie=star-wars-episode-viii-the-last-jedi"
}

partial_url = "http://transcripts.wikia.com/api.php?action=query&prop=revisions&titles={0}&rvprop=content&format=json"

starwars_json = {"data": {"movies": defaultdict(dict), "series": defaultdict(dict)},
               "failed": {"movies": {}, "series": {}}}
for movie in urls.keys():
    print movie
    query = urls[movie]
    try:
        response = urllib2.urlopen(query, timeout=2)
        source = response.read()
        
        # Load the response as .json and extract the relevant information into wikimarkup
        content = source.split("""<div class="scrolling-script-container">""")[1].split("</div>")[0]
        
        no_html = re.compile(r"<[^>]+>")
        text = no_html.sub("", content)

        starwars_json["data"]["movies"][movie] = {"transcript": text}
    
    except Exception as e:
        print "Unexpected Failure During Movie Processing"
        print e
        print query

print "DONE PROCESSING MOVIES"
print        
print starwars_json["data"]["movies"].keys()

# with open("films.json", 'w') as f:
#     json.dumps(movies_json, f)

Star Wars: Episode II Attack of the Clones
Star Wars: The Clone Wars (film)
Star Wars: Episode VII The Force Awakens
Star Wars: Episode III Revenge of the Sith
Star Wars: Episode V The Empire Strikes Back
Rogue One: A Star Wars Story
Solo: A Star Wars Story
Star Wars: Episode IV A New Hope
Star Wars: Episode VIII The Last Jedi
Star Wars: Episode VI Return of the Jedi
Star Wars: Episode I The Phantom Menice
DONE PROCESSING MOVIES

['Star Wars: Episode II Attack of the Clones', 'Star Wars: The Clone Wars (film)', 'Solo: A Star Wars Story', 'Star Wars: Episode VII The Force Awakens', 'Rogue One: A Star Wars Story', 'Star Wars: Episode VIII The Last Jedi', 'Star Wars: Episode V The Empire Strikes Back', 'Star Wars: Episode III Revenge of the Sith', 'Star Wars: Episode VI Return of the Jedi', 'Star Wars: Episode I The Phantom Menice', 'Star Wars: Episode IV A New Hope']


In [28]:
# animated_json = {"data": {"movies": defaultdict(dict), "series": defaultdict(dict), "characters": defaultdict(dict)},
#                "failed": {"movies": {}, "series": {}, "characters": {}}}

# animated_movies = ["Star Wars: The Clone Wars (film)"]

animated_series_names = ["Star Wars: The Clone Wars (series)", "Star Wars Rebels"]

animated_queries = {"Star Wars: The Clone Wars (series)": ["https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show=star-wars-the-clone-wars",
                                                           ["Season 1", "Season 2",
                                                            "Season 3", "Season 4",
                                                            "Season 5", "Season 6"]],
                   "Star Wars Rebels": ["https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show=star-wars-rebels-2014", ["Season 1", "Season 2", "Season 3", "Season 4"]]}

print "STARTING TO PROCESS SERIES AT {0}".format(dt.datetime.now())
for series in animated_series_names:
    print series
    series_query = animated_queries[series][0]
    series_response = urllib2.urlopen(series_query)
    series_source = series_response.read()
    seasons = series_source.split("""<div class="season-episodes">""")[1:]
    
    for season in seasons:
        season = season.split("</div>")[0]
        soup = BeautifulSoup(season)
        episode_list = {}
        for link in soup.findAll('a'):
            episode_list[" ".join(link.contents[0].split(" ")[1:])]=link.get('href')

        for episode in episode_list.keys():
            query = "https://www.springfieldspringfield.co.uk/{0}".format(episode_list[episode])
            try:
                response = urllib2.urlopen(query, timeout=2)
                source = response.read()

                # Load the response as .json and extract the relevant information into wikimarkup
                content = source.split("""<div class="scrolling-script-container">""")[1].split("</div>")[0]

                no_html = re.compile(r"<[^>]+>")
                text = no_html.sub("", content)
                episode_nr = episode_list[episode].split("episode=")[1]
                if not episode_nr.startswith("s00"):
                    starwars_json["data"]["series"][series][episode] = {"transcript": text, "episode": episode_nr}
            except Exception as e:
                print e
        
print starwars_json["data"]["series"].keys()

STARTING TO PROCESS SERIES AT 2018-11-14 22:49:25.294000
Star Wars: The Clone Wars (series)
Star Wars Rebels
['Star Wars Rebels', 'Star Wars: The Clone Wars (series)']


In [31]:
print starwars_json["data"]["series"]['Star Wars: The Clone Wars (series)'].keys()

[u'Deception', u'Revival', u'Rising Malevolence', u'The Zillo Beast', u'The Gungan General', u'An Old Friend', u'Sabotage', u'Defenders of Peace', u'Nightsisters', u'The Jedi Who Knew Too Much', u'Ghosts of Mortis', u'Legacy of Terror', u'Lightsaber Lost', u'Landing at Point Rain', u'A Necessary Bond', u'Blue Shadow Virus', u'Assassin', u'The Rise Of Clovis', u'Slaves of the Republic', u'Pursuit of Peace', u'Dooku Captured', u'Crisis At The Heart', u'Shadow Warrior', u'Crisis on Naboo', u'Destiny', u'The Citadel', u'Duel of the Droids', u'Grievous Intrigue / The Deserter', u'The Disappeared (2)', u'R2 Come Home / Lethal Trackdown', u'Hostage Crisis', u'The Lawless', u'The Lair of General Grievous', u'Children of the Force', u'Nomad Droids', u'Tipping Points', u'Sacrifice', u'Escape from Kadavo', u'Hunt for Ziro', u'Clone Cadets / ARC Troopers', u'The Disappeared (1)', u'The Lost One', u'Gungan Attack', u'Point of No Return', u'The Box', u'Front Runners', u'Evil Plans', u'Cat and Mouse'

In [32]:
with open("starwars_transcripts_springfield.json", 'w') as f:
    json.dump(starwars_json, f)